In [ ]:
#model 1

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import warnings
from sklearn.preprocessing import OneHotEncoder
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
n_splits = 5
seed = 42
path = 'E:/BDCI Match/'

traindataFile = u"E:/BDCI Match/train_preproceed.csv"
testFile = u"E:/BDCI Match/test.zip"

z = zipfile.ZipFile(testFile)

df1 = pd.read_csv(z.open(z.namelist()[0]))

df = pd.read_csv(traindataFile)
# df = df[["service_type","current_service"]]

#对预测标签重新编码
le = LabelEncoder()
#获取预测结果的所有取值
res_labels = df["current_service"].unique()

le.fit(res_labels)
df["current_service"] = le.transform(df["current_service"])

# 构造原始数据
train = df
test = df1

y = train.pop('current_service')
train_id = train.pop('user_id')

X = train
train_col = train.columns

for i in train_col:
    X_test[i] = X_test[i].replace("\\N",-1)

X_test = test[train_col]
test_id = test['user_id']

X,y,X_test = X.values,y,X_test.values


# lgb 参数
params={
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 15,
    'metric': {'rmse'},  #'metric': {'l2', 'auc'},
    'num_leaves':39,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'num_iterations': 10000,
    'max_depth': -1,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "silent":True
}


# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(15, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

xx_score = []
cv_pred = []

skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
for index,(train_index,test_index) in enumerate(skf.split(X,y)):
    print(index)

    X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    validation_data = lgb.Dataset(X_valid, label=y_valid)

    clf=lgb.train(params,train_data,num_boost_round=100000,valid_sets=[validation_data],early_stopping_rounds=50,feval=f1_score_vali,verbose_eval=1)

    xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

    xx_pred = [np.argmax(x) for x in xx_pred]

    xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

    y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

    y_test = [np.argmax(x) for x in y_test]

    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

# 投票
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id.unique())
df_test['predict'] = submit
df_test['predict'] = list(le.inverse_transform(df_test['predict']))

df_test.to_csv("E:/BDCI Match/sample_submission.csv",index=False)


0
